In [24]:
import csv
import pandas as pd
import re
import string
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vmakh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# define function to preprocess the messages
def preprocess_message(message):
    # remove semicolons and quotes
    message = re.sub('[;"]+', '', message)

    # convert to lowercase
    message = message.lower()

    # remove punctuation
    message = message.translate(str.maketrans('', '', string.punctuation))

    # remove stopwords
    stop_words = set(stopwords.words('russian')) 
    message_tokens = message.split()
    filtered_tokens = [token for token in message_tokens if token not in stop_words]
    message = ' '.join(filtered_tokens)

    return message

# read the csv file
df = pd.read_csv('negative.csv', sep=';', header=None, usecols=[3, 4], names=['message', 'label'])



In [26]:
df.head()

,message,label
0,на работе был полный пиддес :| и так каждое за...,-1
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1
2,@elina_4post как говорят обещаного три года жд...,-1
3,"Желаю хорошего полёта и удачной посадки,я буду...",-1
4,"Обновил за каким-то лешим surf, теперь не рабо...",-1


In [27]:
# preprocess the messages
df['message'] = df['message'].apply(preprocess_message)
df.head()

,message,label
0,работе полный пиддес каждое закрытие месяца св...,-1
1,коллеги сидят рубятся urban terror изза долбан...,-1
2,elina4post говорят обещаного года ждут,-1
3,желаю хорошего полёта удачной посадкия буду оч...,-1
4,обновил какимто лешим surf работает простоплеер,-1


In [11]:
# tokenize the messages
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['message'])
word_index = tokenizer.word_index

# convert the messages to sequences of integers
sequences = tokenizer.texts_to_sequences(df['message'])

# pad the sequences to have the same length
padded_sequences = pad_sequences(sequences, maxlen=100)

# convert the labels to categorical
labels = to_categorical(df['label'], num_classes=2)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# build the model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=100))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_test, y_test))

# evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# save the model
model.save('my_nlp_model.h5')


Epoch 1/3
1400/1400 [==============================] - 2016s 1s/step - loss: 0.0037 - accuracy: 0.9998 - val_loss: 1.4353e-09 - val_accuracy: 1.0000
Epoch 2/3
1400/1400 [==============================] - 2415s 2s/step - loss: 4.5996e-06 - accuracy: 1.0000 - val_loss: 5.6573e-11 - val_accuracy: 1.0000
Epoch 3/3
700/700 [==============================] - 141s 202ms/step - loss: 1.1480e-12 - accuracy: 1.0000
Test loss: 1.1480152765919183e-12
Test accuracy: 1.0


## **Задания**

In [40]:
# split the data into training and testing sets
X=df['message']
labels=df['label']
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


In [41]:
y_train.head()

91825   -1
86999   -1
86326   -1
24572   -1
58539   -1
Name: label, dtype: int64

In [42]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

# выделение токенов с высокой частотой
high_freq_vec = CountVectorizer(max_features=5000)
X_train_high = high_freq_vec.fit_transform(X_train)

# выделение токенов со средней частотой
mid_freq_vec = CountVectorizer(max_features=5000, min_df=5)
X_train_mid = mid_freq_vec.fit_transform(X_train)

# выделение токенов с низкой частотой
low_freq_vec = CountVectorizer(max_features=5000, min_df=2)
X_train_low = low_freq_vec.fit_transform(X_train)


In [43]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# обучение классификаторов
clf_high = MultinomialNB().fit(X_train_high, y_train)
clf_mid = MultinomialNB().fit(X_train_mid, y_train)
clf_low = MultinomialNB().fit(X_train_low, y_train)

# получение предсказаний и вывод отчета классификации
y_pred_high = clf_high.predict(high_freq_vec.transform(X_test))
y_pred_mid = clf_mid.predict(mid_freq_vec.transform(X_test))
y_pred_low = clf_low.predict(low_freq_vec.transform(X_test))

print("High frequency tokens:\n", classification_report(y_test, y_pred_high))
print("Mid frequency tokens:\n", classification_report(y_test, y_pred_mid))
print("Low frequency tokens:\n", classification_report(y_test, y_pred_low))


High frequency tokens:
               precision    recall  f1-score   support

          -1       1.00      1.00      1.00     22385

    accuracy                           1.00     22385
   macro avg       1.00      1.00      1.00     22385
weighted avg       1.00      1.00      1.00     22385

Mid frequency tokens:
               precision    recall  f1-score   support

          -1       1.00      1.00      1.00     22385

    accuracy                           1.00     22385
   macro avg       1.00      1.00      1.00     22385
weighted avg       1.00      1.00      1.00     22385

Low frequency tokens:
               precision    recall  f1-score   support

          -1       1.00      1.00      1.00     22385

    accuracy                           1.00     22385
   macro avg       1.00      1.00      1.00     22385
weighted avg       1.00      1.00      1.00     22385



In [45]:
# Задание 2

# определение наиболее важных токенов для каждого класса
tokens = high_freq_vec.get_feature_names_out()
high_prob = np.exp(clf_high.feature_log_prob_)
mid_prob = np.exp(clf_mid.feature_log_prob_)
low_prob = np.exp(clf_low.feature_log_prob_)

high_top_indices = np.argsort(high_prob[0])[::-1][:10]
mid_top_indices = np.argsort(mid_prob[0])[::-1][:10]
low_top_indices = np.argsort(low_prob[0])[::-1][:10]

print("Наиболее важные токены для классификатора, обученного на токенах с высокой частотой:")
print([tokens[i] for i in high_top_indices])
print()
print("Наиболее важные токены для классификатора, обученного на токенах со средней частотой:")
print([tokens[i] for i in mid_top_indices])
print()
print("Наиболее важные токены для классификатора, обученного на токенах с низкой частотой:")
print([tokens[i] for i in low_top_indices])

Наиболее важные токены для классификатора, обученного на токенах с высокой частотой:
['rt', 'это', 'хочу', 'сегодня', 'очень', 'день', 'завтра', 'могу', 'просто', 'блин']

Наиболее важные токены для классификатора, обученного на токенах со средней частотой:
['seconds', 'этими', 'хотят', 'сдохла', 'очередь', 'делают', 'забываю', 'можешь', 'просыпаться', 'блииин']

Наиболее важные токены для классификатора, обученного на токенах с низкой частотой:
['seconds', 'этих', 'хотят', 'сдохну', 'ощущение', 'делом', 'забыть', 'мокрая', 'просыпается', 'блииин']
